In [7]:
import nltk

In [8]:
import wikipediaapi

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag, FreqDist, download
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB  # Add this import
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag, FreqDist, download
import numpy as np
import requests
from bs4 import BeautifulSoup
import wikipediaapi


In [14]:
wiki_wiki = wikipediaapi.Wikipedia('english')

# Function to retrieve a text from a Wikipedia page
def get_wikipedia_text(page_title):
    page = wiki_wiki.page(page_title)
    
    if not page.exists():
        return None
    
    return page.text

# Function to retrieve text from a website
def get_website_text(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')  # You may need to adjust this based on the HTML structure of the website
        text = ' '.join([paragraph.get_text() for paragraph in paragraphs])
        return text
    else:
        return None

def extract_keywords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return filtered_words

def extract_nouns(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    tagged_words = pos_tag(words)
    nouns = [word.lower() for word, pos in tagged_words if pos.startswith('N') and word.lower() not in stop_words and word.isalnum()]
    return nouns

def extract_top_nouns(topics, num_top_nouns=10):
    all_nouns = []

    for topic in topics:
        text = get_wikipedia_text(topic)
        if text:
            all_nouns.extend(extract_nouns(text))

    nouns_freq_dist = FreqDist(all_nouns)
    top_nouns = [word for word, _ in nouns_freq_dist.most_common(num_top_nouns)]

    return top_nouns

# Sample annotated keywords for geographical and non-geographical topics
geographical_topics = ['geography', 'GIS', 'map', 'Geoprocessing', 'surface','sun','wind', 'ocean','water']
non_geographical_topics = ['programming', 'technology', 'history', 'medical', 'estimates','behave','physic', 'economy']

top_geographical_keywords = extract_top_nouns(geographical_topics, num_top_nouns=10)
top_non_geographical_keywords = extract_top_nouns(non_geographical_topics, num_top_nouns=10)

all_topics = geographical_topics + non_geographical_topics
all_docs = []
all_labels = []

for topic in all_topics:
    text = get_wikipedia_text(topic)
    if text:
        nouns = extract_nouns(text)
        all_docs.append(" ".join(nouns))
        all_labels.append(1 if topic in geographical_topics else 0)

all_top_keywords = top_geographical_keywords + top_non_geographical_keywords
vectorizer = TfidfVectorizer(vocabulary=all_top_keywords)
X = vectorizer.fit_transform(all_docs)
y = all_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression classifier
logistic_classifier = LogisticRegression()
logistic_classifier.fit(X_train, y_train)
logistic_predictions = logistic_classifier.predict(X_test)

# Evaluate accuracy
accuracy_logistic = accuracy_score(y_test, logistic_predictions)

# Replace the URL with the website you want to classify
website_url = "https://en.wikipedia.org/wiki/Cinematography"
website_text = get_website_text(website_url)

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train, y_train)
naive_bayes_predictions = naive_bayes_classifier.predict(X_test)

# Evaluate accuracy, precision, and recall
accuracy_naive_bayes = accuracy_score(y_test, naive_bayes_predictions)
precision_naive_bayes = precision_score(y_test, naive_bayes_predictions)
recall_naive_bayes = recall_score(y_test, naive_bayes_predictions)

# Print the results
print(f"Accuracy (Naïve Bayes): {accuracy_naive_bayes}")
print(f"Precision (Naïve Bayes): {precision_naive_bayes}")
print(f"Recall (Naïve Bayes): {recall_naive_bayes}")

if website_text:
    # Vectorize the website text using TF-IDF
    website_vectorized = vectorizer.transform([" ".join(extract_nouns(website_text))])

    # Logistic Regression prediction using TF-IDF
    logistic_website_prediction = logistic_classifier.predict(website_vectorized)

    if logistic_website_prediction == 1:
        print(f'The content of the website "{website_url}" is geographical according to Logistic Regression with TF-IDF.')
    else:
        print(f'The content of the website "{website_url}" is non-geographical according to Logistic Regression with TF-IDF.')
else:
    print(f'Unable to fetch text from the website "{website_url}". Please check the URL or try another website.')

Accuracy (Naïve Bayes): 1.0
Precision (Naïve Bayes): 1.0
Recall (Naïve Bayes): 1.0
The content of the website "https://en.wikipedia.org/wiki/Cinematography" is non-geographical according to Logistic Regression with TF-IDF.
